### Prompt Chaining
Prompt chaining is a technique in NLP where multiple prompts are sequenced to gether to guide a model through a complex task or reasing process. Instead of relaying on single prompt to achieve a desired outcome, prompt chaining breaks the task into smaller, managable steps

#### How Prompt Chaining Works with LangGraph
1. Define the Task: Start by breaking down the problem into smaller sub-tasks. For example, if you want to generate a detailed report, you might split it into steps like "gather data," "analyze data," and "write summary."

2. Create Nodes: Each sub-task becomes a node in the LangGraph structure. A node could be a prompt that instructs the model to perform a specific action, such as "List key facts about X" or "Summarize the following text."

3. Establish Edges: Edges define the sequence and dependencies between nodes. For instance, the output of the "gather data" node flows into the "analyze data" node, ensuring the model has the necessary context to proceed.

4. Execute the Graph: LangGraph processes the nodes in order, passing information along the edges. The model generates responses step-by-step, refining the output as it progresses through the chain.

5. Iterate if Needed: LangGraph supports conditional logic and loops, so you can revisit earlier nodes or adjust the flow based on intermediate results.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm= ChatGroq(model="llama-3.1-8b-instant")

result=llm.invoke("Hello")
result

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

#Graph State

class State(TypedDict):
    topic:str
    story:str
    improved_story:str
    final_story:str

#nodes

def generate_story(state:State):
    msg=llm.invoke(f"write a one sentence story about {state["topic"]}")
    return{"story":msg.content}

def check_conflict(state:State):
    if "?" in state["story"] or "!" in state["story"]:
        return "Fail"
    return "Pass"
def improved_story(state:State):
    msg=llm.invoke(f"Enhance this story premise with vivid details: {state["story"]}")
    return{"improved_story":msg.content}
def polish_story(state:State):
    msg=llm.invoke(f"Add an unexpected twist to this story premise:{state["improved_story"]}")
    return{"final_story":msg.content}

In [ ]:
#Build the graph

graph=StateGraph(State)

graph.add_node("generate",generate_story)
graph.add_node("improve",improved_story)
graph.add_node("polish",polish_story)

#define the edges

graph.add_edge(START,"generate")
graph.add_conditional_edges("generate",check_conflict,{"Pass":"improve","Fail":"generate"})
graph.add_edge("improve","polish")
graph.add_edge("polish",END)

#compile the graph

compiled_graph=graph.compile()

graph_image=compiled_graph.get_graph().draw_mermaid_png()
display(Image(graph_image))

In [ ]:
state={"topic":"Agentic AI Systems"}
result=compiled_graph.invoke(state)
result